In [ ]:
import torch
import torch.linalg as tla
import numpy as np
import matplotlib.pyplot as plt
import numml.sparse as sp

In [ ]:
# Create our favorite poisson operator

N = 16
A = sp.eye(N)*2. - sp.eye(N, k=1) - sp.eye(N, k=-1)

In [ ]:
Dinv = sp.diag(1./A.diagonal())

In [ ]:
def err_prop_1(omega):
    G = sp.eye(N) - sp.diag(omega) @ Dinv @ A
    return (G.T@G).trace()

def err_prop_2(omega):
    G = sp.eye(N) - sp.diag(omega) @ A
    return (G.T@G).trace()

In [ ]:
omega = torch.ones(N, requires_grad=True)

optimizer = torch.optim.Adam([omega], lr=1e-2)
batch = 20
lh = []
oh = []

for i in range(100):
    optimizer.zero_grad()
    
    loss = err_prop_1(omega)
    loss.backward()
    
    optimizer.step()
    
    lh.append(loss.item())
    oh.append(omega.detach().numpy().copy())
    if i % 10 == 0:
        print(i, 'loss:', round(loss.item(), 3), 'omega:', torch.round(omega.detach(), decimals=2))

Lh = np.array(lh)
Oh = np.array(oh)

plt.figure(figsize=(6, 5))

plt.figure()
plt.plot(Lh, 'k')
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.grid()

ax2 = plt.gca().twinx()
ax2.plot(Oh)
ax2.set_ylabel('Jacobi Weight')

plt.figure(figsize=(6, 3))
plt.title('Entry-wise Jacobi weights (w/ D^{-1})')
plt.plot(omega.detach(), 'o-')
plt.ylim(0.3, 1)
plt.xlabel('Node')
plt.ylabel('Jacobi Weight')
plt.grid()

In [ ]:
omega = torch.ones(N, requires_grad=True)
with torch.no_grad():
    omega /= 2.

optimizer = torch.optim.Adam([omega], lr=1e-2)
batch = 20
lh = []
oh = []

for i in range(100):
    optimizer.zero_grad()
    
    loss = err_prop_2(omega)
    loss.backward()
    
    optimizer.step()
    
    lh.append(loss.item())
    oh.append(omega.detach().numpy().copy())
    if i % 10 == 0:
        print(i, 'loss:', round(loss.item(), 3), 'omega:', torch.round(omega.detach(), decimals=2))

Lh = np.array(lh)
Oh = np.array(oh)

plt.figure(figsize=(6, 5))

plt.figure()
plt.plot(Lh, 'k')
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.grid()

ax2 = plt.gca().twinx()
ax2.plot(Oh)
ax2.set_ylabel('Jacobi Weight')

plt.figure(figsize=(6, 3))
plt.title('Entry-wise Jacobi weights (w/o D^{-1})')
plt.plot(omega.detach(), 'o-')
plt.ylim(0.3, 1)
plt.xlabel('Node')
plt.ylabel('Jacobi Weight')
plt.grid()